# **2. Baseline**
In the first place, we have tried to define a baseline to better understand the dataset potentiality and which value of MAPE we can expect to achive. <br>
For the baseline we have tried to use the sales of the previous week as our prediction.

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import sys
sys.path.append("../")

csv_train = pd.read_csv("../dataset/original/train.csv")
csv_test = pd.read_csv("../dataset/original/x_test.csv")

In [5]:
from preprocessing.preprocessing import preprocessing, train_validation_split
df = preprocessing(csv_train, csv_test, useTest=False)
df_t = df[~df.target.isna()]
df_train, df_val, val_dates = train_validation_split(df_t, k=0.20)
df_val

,Date,sku,pack,size (GM),brand,price,POS_exposed w-1,volume_on_promo w-1,sales w-1,scope,target,real_target
4663,2018-12-29,2682,0,105.44,0,1.73,1.0,23.505295,9.623178,0,9.138952,9310.0
2801,2018-12-29,1600,1,878.68,2,1.55,0.0,0.000000,10.600178,0,10.189456,26620.0
423,2018-12-29,549,1,114.23,1,0.56,2.0,27.355566,10.832635,1,9.828441,18553.0
3333,2018-12-29,1633,1,131.80,2,0.51,0.0,15.380149,10.076095,0,9.474933,13028.0
739,2018-12-29,686,0,125.65,3,3.49,0.0,0.000000,10.043684,1,9.491602,13247.0
...,...,...,...,...,...,...,...,...,...,...,...,...
4289,2019-06-22,2410,1,395.41,4,1.72,3.0,100.000000,9.423838,0,9.030855,8356.0
2693,2019-06-22,1554,1,395.41,2,1.00,0.0,0.000000,10.767116,0,10.724654,45462.0
3624,2019-06-22,2249,1,395.41,4,1.73,3.0,93.796134,9.339701,0,8.936956,7607.0
1712,2019-06-22,1065,1,125.65,3,0.56,3.0,36.370096,11.621457,1,11.554048,104197.0


In [34]:
df_val = df_val.sort_values(['sku', 'Date'])
predictions = []

for s in sorted(set(df_val.sku)):
    current_sku_pred = []
    current_sku_pred.append(df_train[df_train.sku == s]['real_target'].tail(1).values.squeeze())    
    current_sku_pred.extend(df_val[df_val.sku == s]['real_target'].shift(1)[1:])
    predictions.extend(current_sku_pred)

df_val['predictions'] = predictions
df_val

,Date,sku,pack,size (GM),brand,price,POS_exposed w-1,volume_on_promo w-1,sales w-1,scope,target,real_target,predictions
107,2018-12-29,144,0,114.23,1,3.50,0.0,0.043608,9.871894,1,9.293762,10869.0,19377.0
108,2019-01-05,144,0,114.23,1,3.49,0.0,1.513019,9.293762,1,9.640303,15371.0,10869
109,2019-01-12,144,0,114.23,1,1.79,0.0,0.000000,9.640303,1,11.573484,106242.0,15371
110,2019-01-19,144,0,114.23,1,1.75,57.0,99.881105,11.573484,1,11.664109,116320.0,106242
111,2019-01-26,144,0,114.23,1,1.75,58.0,99.792827,11.664109,1,11.755236,127418.0,116320
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6014,2019-05-25,2718,1,395.41,0,1.11,0.0,26.050480,10.430462,0,10.414183,33328.0,33875
6015,2019-06-01,2718,1,395.41,0,1.30,1.0,43.099496,10.414183,0,10.021848,22512.0,33328
6016,2019-06-08,2718,1,395.41,0,1.55,0.0,0.000000,10.021848,0,9.767782,17461.0,22512
6017,2019-06-15,2718,1,395.41,0,1.55,0.0,0.000000,9.767782,0,9.747185,17105.0,17461


In [39]:
from metrics.MAPE import MAPE
print("Baseline validation MAPE: %f" % MAPE(df_val['real_target'], df_val['predictions']))

Baseline validation MAPE: 19.212902


This is the result on the validation of our baseline, a very simple model which can be surely improved. Let's try what is the MAPE on test. 

In [43]:
df = preprocessing(csv_train, csv_test)
df = df.sort_values(['Date'])
df_test = df[df.Date >= '2019-06-29']
df_test

,Date,sku,pack,size (GM),brand,price,POS_exposed w-1,volume_on_promo w-1,sales w-1,scope,target,real_target
1871,2019-06-29,1206,1,125.65,3,0.56,6.0,22.049205,10.753318,1,10.659375,42589.0
133,2019-06-29,144,0,114.23,1,3.20,0.0,1.249144,9.555348,1,9.694555,16228.0
291,2019-06-29,546,1,114.23,1,0.56,3.0,39.470552,11.171547,1,11.025295,61407.0
1081,2019-06-29,1027,1,114.23,1,0.56,10.0,24.386541,11.102518,1,10.819638,49992.0
1239,2019-06-29,1035,1,114.23,1,0.56,0.0,20.875194,10.703110,1,10.458636,34843.0
...,...,...,...,...,...,...,...,...,...,...,...,...
947,2019-12-14,688,1,125.65,3,0.56,0.0,32.072577,10.393784,1,NaN,NaN
1105,2019-12-14,1027,1,114.23,1,0.56,0.0,26.742559,10.770840,1,NaN,NaN
1263,2019-12-14,1035,1,114.23,1,0.55,6.0,36.060336,10.547655,1,NaN,NaN
789,2019-12-14,686,0,125.65,3,3.01,0.0,95.954144,10.626072,1,NaN,NaN


In [52]:
df_test = df_test.sort_values(['sku', 'Date'])
predictions = []

for s in sorted(set(df_test.sku)):
    current_sku_pred = []
    current_sku_pred.append(df[df.sku == s][df.Date == '2019-06-22']['real_target'].values.squeeze())    
    current_sku_pred.extend(df_test[df_test.sku == s]['real_target'].shift(1)[1:])
    predictions.extend(current_sku_pred)

df_test['predictions'] = predictions
df_test

C:\Users\david\anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


,Date,sku,pack,size (GM),brand,price,POS_exposed w-1,volume_on_promo w-1,sales w-1,scope,target,real_target,predictions
133,2019-06-29,144,0,114.23,1,3.20,0.0,1.249144,9.555348,1,9.694555,16228.0,14119.0
134,2019-07-06,144,0,114.23,1,3.22,0.0,0.000000,9.694555,1,9.804496,18114.0,16228
135,2019-07-13,144,0,114.23,1,3.02,0.0,0.000000,9.804496,1,9.838896,18748.0,18114
136,2019-07-20,144,0,114.23,1,3.25,0.0,32.730496,9.838896,1,9.788862,17833.0,18748
137,2019-07-27,144,0,114.23,1,2.99,0.0,0.069635,9.788862,1,9.747185,17105.0,17833
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1890,2019-11-09,1206,1,125.65,3,0.32,11.0,98.734224,11.632263,1,11.620470,111353.0,112674
1891,2019-11-16,1206,1,125.65,3,0.41,19.0,98.800841,11.620470,1,11.056588,63359.0,111353
1892,2019-11-23,1206,1,125.65,3,0.57,2.0,99.129248,11.056588,1,10.469909,35238.0,63359
1893,2019-11-30,1206,1,125.65,3,0.57,0.0,28.696578,10.469909,1,10.441237,34242.0,35238


In [53]:
df_test = df_test.drop(df_test[df_test.Date == '2019-12-14'].index)
print("Baseline test MAPE: %f" % MAPE(df_test['real_target'], df_test['predictions']))

Baseline test MAPE: 27.522491


As we can see the MAPE is very bad, but at least we have a sort of landmark. What we can do now is starting to better grasp the behaviour of the time series in the dataset and begin to deeply analyze the data.